## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression

import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [2]:
data0 = pd.read_csv('geo_data_0.csv', index_col=[0])
data1 = pd.read_csv('geo_data_1.csv', index_col=[0])
data2 = pd.read_csv('geo_data_2.csv', index_col=[0])

data0.info()
display(data0.head())
print(data0.duplicated().sum())
print()

data1.info()
display(data1.head())
print(data1.duplicated().sum())
print()

data2.info()
display(data2.head())
print(data2.duplicated().sum())
print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


0



Выборки равного объема, по 100 тыс объектов в каждой. Дубликатов нет. Первый столбец удалим, поскольку он не влияет на объемы запасов в скважинах и может только "сбить" модель. Также заметно, что признак f2 имеет другой масштаб, поэтому для улучшения качества модели проведем масштабирование. Но сперва построим матрицы корреляции:

In [3]:
print(data0.corr())
print()
print(data1.corr())
print()
print(data2.corr())

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000

               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000

               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


Отметим, что признак f2 наиболее сильно влияет на объемы запасов. При этом во втором регионе корреляция практически полная, для 1го и 3го регионов коэффициент составляет всего 0,45-0,5. Важно подчеркнуть, что f0 и f1 не имеют существенного влияния на целевой показатель, хоть какая-то зависимость наблюдается только в 1ом регионе.

In [4]:
#Подготовка данных региона 1
data0 = data0.drop('id', axis = 1)
display(data0.head())
target0 = data0['product']
features0 = data0.drop(['product'], axis = 1)
display(features0.head())
display(target0.head())

#Разбивка на обучающую и валидационную выборки
features0_train, features0_valid, target0_train, target0_valid = train_test_split(features0, target0, test_size = 0.25, random_state = 12345)
print(target0_train.shape)
print(features0_train.shape)
print(target0_valid.shape)
print(features0_valid.shape)

#Масштабирование признаков
scaler0 = StandardScaler()
scaler0.fit(features0[features0.columns])
features0_train[features0.columns] = scaler0.transform(features0_train[features0.columns])
features0_valid[features0.columns] = scaler0.transform(features0_valid[features0.columns])



,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2
0,0.705745,-0.497823,1.221170
1,1.334711,-0.340164,4.365080
2,1.022732,0.151990,1.419926
3,-0.032172,0.139033,2.978566
4,1.988431,0.155413,4.751769


0    105.280062
1     73.037750
2     85.265647
3    168.620776
4    154.036647
Name: product, dtype: float64

(75000,)
(75000, 3)
(25000,)
(25000, 3)


In [5]:
#Подготовка данных региона 2
data1 = data1.drop('id', axis = 1)
display(data1.head())
target1 = data1['product']
features1 = data1.drop(['product'], axis = 1)
display(features1.head())
display(target1.head())

#Разбивка на обучающую и валидационную выборки
features1_train, features1_valid, target1_train, target1_valid = train_test_split(features1, target1, test_size = 0.25, random_state = 12345)
print(target1_train.shape)
print(features1_train.shape)
print(target1_valid.shape)
print(features1_valid.shape)

#Масштабирование признаков
scaler1 = StandardScaler()
scaler1.fit(features1[features1.columns])
features1_train[features1.columns] = scaler1.transform(features1_train[features1.columns])
features1_valid[features1.columns] = scaler1.transform(features1_valid[features1.columns])



,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2
0,-15.001348,-8.276000,-0.005876
1,14.272088,-3.475083,0.999183
2,6.263187,-5.948386,5.001160
3,-13.081196,-11.506057,4.999415
4,12.702195,-8.147433,5.004363


0      3.179103
1     26.953261
2    134.766305
3    137.945408
4    134.766305
Name: product, dtype: float64

(75000,)
(75000, 3)
(25000,)
(25000, 3)


In [6]:
#Подготовка данных региона 3
data2 = data2.drop('id', axis = 1)
display(data2.head())
target2 = data2['product']
features2 = data2.drop('product', axis = 1)
display(features2.head())
display(target2.head())

#Разбивка на обучающую и валидационную выборки
features2_train, features2_valid, target2_train, target2_valid = train_test_split(features2, target2, test_size = 0.25, random_state = 12345)
print(target2_train.shape)
print(features2_train.shape)
print(target2_valid.shape)
print(features2_valid.shape)

#Масштабирование признаков
scaler2 = StandardScaler()
scaler2.fit(features2[features2.columns])
features2_train[features2.columns] = scaler2.transform(features2_train[features2.columns])
features2_valid[features2.columns] = scaler2.transform(features2_valid[features2.columns])

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2
0,-1.146987,0.963328,-0.828965
1,0.262778,0.269839,-2.530187
2,0.194587,0.289035,-5.586433
3,2.236060,-0.553760,0.930038
4,-0.515993,1.716266,5.899011


0     27.758673
1     56.069697
2     62.871910
3    114.572842
4    149.600746
Name: product, dtype: float64

(75000,)
(75000, 3)
(25000,)
(25000, 3)


Итак, данные по всем 3 регионам готовы для обучения

## Обучение и проверка модели

In [7]:
#Регион 1
model0 = LinearRegression()
model0.fit(features0_train, target0_train)
predictions0_valid = model0.predict(features0_valid)
predictions0_valid = pd.Series(predictions0_valid)
print
print('Регион 1')
print('Средний запас сырья в скважине составляет', predictions0_valid.mean(), 'тыс. баррелей')
print('rmse модели составляет', mean_squared_error(target0_valid, predictions0_valid) ** 0.5)
print()

#Регион 2
model1 = LinearRegression()
model1.fit(features1_train, target1_train)
predictions1_valid = model1.predict(features1_valid)
predictions1_valid = pd.Series(predictions1_valid)
print('Регион 2')
print('Средний запас сырья в скважине составляет', predictions1_valid.mean(), 'тыс. баррелей')
print('rmse модели составляет', mean_squared_error(target1_valid, predictions1_valid) ** 0.5)
print()

#Регион 3
model2 = LinearRegression()
model2.fit(features2_train, target2_train)
predictions2_valid = model2.predict(features2_valid)
predictions2_valid = pd.Series(predictions2_valid)
print('Регион 3')
print('Средний запас сырья в скважине составляет', predictions2_valid.mean(), 'тыс. баррелей')
print('rmse модели составляет', mean_squared_error(target2_valid, predictions2_valid) ** 0.5)


Регион 1
Средний запас сырья в скважине составляет 92.59256778438035 тыс. баррелей
rmse модели составляет 37.5794217150813

Регион 2
Средний запас сырья в скважине составляет 68.72854689544602 тыс. баррелей
rmse модели составляет 0.8930992867756171

Регион 3
Средний запас сырья в скважине составляет 94.96504596800489 тыс. баррелей
rmse модели составляет 40.02970873393434


Выводы:
По оценке средний запас сырья в скважинах региона 2 существенно (в 1,5 раза) ниже, чем в двух остальных. Высокая корреляция с признаком f2 существенно сказалась ошибке (rmse), которая почти в 40 раз ниже, чем для регионов 1 и 3. Пока наиболее перспективным выглядят регионы 1 и 3, средняя оценка запасов несколько выше чем в 3ем, а ошибка чуть ниже в 1ом.

## Подготовка к расчёту прибыли

In [8]:
total_points = 500
selected_points = 200
total_budget = 10000000 #в тыс. руб
income_per_barrel = 450 #в тыс. руб
max_losses_prob = 0.025

#Расчет запасов в скважине, достаточных для окупаемости
zero_volume = total_budget/(selected_points * income_per_barrel)
print('Безубыточный запас составляет', zero_volume)

Безубыточный запас составляет 111.11111111111111


Расчет безубыточного запаса сырья на одну скважину составляет 111 тыс. баррелей. Ясно, что разработка всех скважин из валидационной выборки по любому из регионов приведет к убытку. Случайный выбор скважин в данном случае ни к чему положительному не приведет, необходимость модели для оценки очевидна.. 

## Расчёт прибыли и рисков 

In [9]:
#проверка
print('0')
print(features0_valid.index)
print()
print('1')

print(predictions0_valid.index)
print()
print('2')
print(target0_valid.index)
print()
# замечаем, что индексы прогнозов обнулились

0
Int64Index([71751, 80493,  2655, 53233, 91141,  9539,  8482, 30284, 66393,
            99584,
            ...
            13095, 89694, 79887, 62617, 99399, 12581, 18456, 73035, 63834,
            43558],
           dtype='int64', length=25000)

1
RangeIndex(start=0, stop=25000, step=1)

2
Int64Index([71751, 80493,  2655, 53233, 91141,  9539,  8482, 30284, 66393,
            99584,
            ...
            13095, 89694, 79887, 62617, 99399, 12581, 18456, 73035, 63834,
            43558],
           dtype='int64', length=25000)



In [10]:
#возвращаем исходные индексы
predictions0_valid.index = features0_valid.index
predictions1_valid.index = features1_valid.index
predictions2_valid.index = features2_valid.index

In [11]:
#Новая функция прибыли:

def total_income(pred_volumes, real_volumes, budget):
    top200 = pred_volumes.sort_values(ascending=False).index
    raw_revenue = income_per_barrel * real_volumes.loc[top200][:200].sum()
    income = raw_revenue - budget
    return income


# Обнуляем переменные для расчетов
state = RandomState(12345)
income0 = []
income1 = []
income2 = []
count0 = 0
count1 = 0
count2 = 0

for i in range(1000):
    sample_0 = target0_valid.sample(n=500, replace = True, random_state = state)
    predicted0_volumes = predictions0_valid.loc[sample_0.index]
    income_sample_0 = total_income(predicted0_volumes, sample_0, total_budget)
    income0.append(income_sample_0)
    if income_sample_0 < 0:
        count0+=1
        
    sample_1 = target1_valid.sample(n=500, replace = True, random_state = state)
    predicted1_volumes = predictions1_valid.loc[sample_1.index]
    income_sample_1 = total_income(predicted1_volumes, sample_1, total_budget)
    income1.append(income_sample_1)
    if income_sample_1 < 0:
        count1+=1
        
    sample_2 = target2_valid.sample(n=500, replace = True, random_state = state)
    predicted2_volumes = predictions2_valid.loc[sample_2.index]
    income_sample_2 = total_income(predicted2_volumes, sample_2, total_budget)
    income2.append(income_sample_2)
    if income_sample_2 < 0:
        count2+=1


        
income0 = pd.Series(income0)
income1 = pd.Series(income1)
income2 = pd.Series(income2)    
    
print('Средняя прибыль в регионе 1, тыс руб:', int(income0.mean()))
print('Доверительный интервал для региона 1: [', income0.quantile(0.025), ';', income0.quantile(0.975), ']')
print('Вероятность убытков для региона 1 составляет {:.2%}'.format(count0/1000))
print()
print('Средняя прибыль в регионе 2, тыс руб:', int(income1.mean()))
print('Доверительный интервал для региона 2: [', income1.quantile(0.025), ';', income1.quantile(0.975), ']')
print('Вероятность убытков для региона 2 составляет {:.2%}'.format(count1/1000))
print()
print('Средняя прибыль в регионе 3, тыс руб:', int(income2.mean()))
print('Доверительный интервал для региона 3: [', income2.quantile(0.025), ';', income2.quantile(0.975), ']')
print('Вероятность убытков для региона 3 составляет {:.2%}'.format(count2/1000))


Средняя прибыль в регионе 1, тыс руб: 414736
Доверительный интервал для региона 1: [ -123528.11644936843 ; 968562.8654960695 ]
Вероятность убытков для региона 1 составляет 7.00%

Средняя прибыль в регионе 2, тыс руб: 513172
Доверительный интервал для региона 2: [ 85329.74622608535 ; 940106.2882984517 ]
Вероятность убытков для региона 2 составляет 0.90%

Средняя прибыль в регионе 3, тыс руб: 410198
Доверительный интервал для региона 3: [ -140423.72725207437 ; 967716.1569820443 ]
Вероятность убытков для региона 3 составляет 8.50%


Главный вывод: "Чем точнее модель (прогноз) - тем точнее выбор скважин - тем уже доверительный интервал - тем проще оценивать риски".

Несмотря на то, что средняя оценка объема ресурсов для скважин региона 2 была минимальной, только по нему можно с уверенностью говорить, что вероятность убытков минимальная (0,9%) и проходит по критерию надежности. И вызвано это тем, что с наибольшей вероятностью мы выбираем именно наиболее "прибыльные" скважины.. Регионы 1 и 3 не проходят по критерию надежности, и как показывают результаты оценки, в среднем дают меньшую прибыль от реализации проекта. 

Стоит отметить, что по 1 и 3 регионам можно задуматься о необходимости введения дополнительных признаков для повышения точности моделей.

